# A Trader Stabilize Fiat Value of Crypto Currency

In [ ]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 60
SAD_COOLING_TIME = 60 * 12
BUY_RATE = 30
BUY_AMOUNT = 0.25
MAX_SELL_COUNT = 32
MAX_USED_JPY = 400000
MIN_UNIT_CC_GAIN_JPY = 250
MIN_UNIT_CC_LOSS_JPY = 50
MAX_UNIT_CC_LOSS_JPY = 5000
MAX_LOSS_JPY = 2000
MAX_TOTAL_LOSS_JPY = 2000

In [ ]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [ ]:
pub = python_bitbankcc.public()

In [ ]:
## Simulated One
from unittest.mock import Mock
prv = Mock()

In [ ]:
# Actual One
import os
prv = python_bitbankcc.private(
    os.environ['BITBANK_API_KEY'],
    os.environ['BITBANK_API_SECRET']
)

In [ ]:
TEST_RATIO = 1

In [ ]:
sample_number = 0
buy_count = 0
sell_count = 0
bought_amount = 0
used_jpy = 0
please_sell = False
total_gained_jpy = 0
with open('sample-history.txt', 'a', 1) as fsh, open('transaction-history.txt', 'a', 1) as fth:
    while True:
        now_ticker = pub.get_ticker('eth_jpy')
        now_sell_jpy = int(now_ticker.get('buy', '0'))
        now_buy_jpy = int(now_ticker.get('sell', '1'))
        # comment out to speed up
        fsh.write(f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n')

        cooling_time = HAPPY_COOLING_TIME
        gained_jpy = now_sell_jpy * bought_amount - used_jpy
        avg_jpy = used_jpy/bought_amount if bought_amount != 0 else sys.maxsize
        if now_sell_jpy > avg_jpy + MIN_UNIT_CC_GAIN_JPY:
            please_sell = True
        if sample_number % BUY_RATE == 0:
            avg_jpy = used_jpy/bought_amount if bought_amount != 0 else -sys.maxsize
            if now_sell_jpy < avg_jpy - MAX_UNIT_CC_LOSS_JPY:
                please_sell = True
                cooling_time = SAD_COOLING_TIME
            if gained_jpy < -MAX_LOSS_JPY:
                please_sell = True
                cooling_time = SAD_COOLING_TIME

        if please_sell:
            prv.order(
                pair='eth_jpy', price=str(now_sell_jpy), amount=str(bought_amount), side='sell', order_type='market'
            )
            total_gained_jpy += gained_jpy
            fth.write(
                f'SOLD {bought_amount} ETH for {now_sell_jpy-avg_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
                f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy} JPY in total at {datetime.now()}\n\n'
            )
            sell_count += 1
            if sell_count == MAX_SELL_COUNT:
                break
            sample_number = 0
            buy_count = 0
            bought_amount = 0
            used_jpy = 0
            please_sell = False
            time.sleep(cooling_time/TEST_RATIO)
            continue

        if sample_number % BUY_RATE == 0:
            buy_count += 1
            bought_amount_after = bought_amount + BUY_AMOUNT
            used_jpy_after = used_jpy + now_buy_jpy * BUY_AMOUNT
            avg_jpy = used_jpy_after / bought_amount_after
            if used_jpy_after <= MAX_USED_JPY:
                prv.order(
                    pair='eth_jpy', price=str(now_buy_jpy), amount=str(BUY_AMOUNT), side='buy', order_type='market'
                )
                fth.write(f'BOUGHT {BUY_AMOUNT} ETH for {now_buy_jpy} JPY: SELL for {now_sell_jpy} JPY: NO.{buy_count} at {datetime.now()}\n')
                bought_amount = bought_amount_after
                used_jpy = used_jpy_after
            please_sell = True if used_jpy_after > MAX_USED_JPY else False

        time.sleep(SAMPLE_INTERVAL/TEST_RATIO)
        sample_number += 1